In [7]:
import pandas as pd

In [75]:
df = pd.read_csv('FINAL_ARXIV_2025_with_affiliations.csv')

In [66]:
import pandas as pd
import os

# 1. Load the titles to search for
print("Loading titles...")
with open('papers_with_missing_affiliations.txt', 'r') as f:
    titles = [line.strip() for line in f if line.strip()]
data = pd.DataFrame(titles, columns=['title'])
print(f"Loaded {len(data)} titles.")

# 2. Parse the latex affiliations file
# Note: Using latex_affiliations_output.txt as primary source
input_file = "latex_affiliations_output.txt"
if not os.path.exists(input_file) and os.path.exists("latex_affiliations_output_2.txt"):
    input_file = "latex_affiliations_output_2.txt"

print(f"Parsing {input_file}... this may take a moment.")
paper_sections = {}
current_paper = None
affiliation_started = False
failed_download_count = 0
papers_with_errors = []

with open(input_file, "r") as f:
    for line in f:
        if line.startswith("PAPER: "):
            current_paper = line.replace("PAPER: ", "").strip()
            paper_sections[current_paper] = []
            affiliation_started = False
        elif line.startswith("ERROR: Failed to download LaTeX sources"):
            if current_paper:
                papers_with_errors.append(current_paper)
                failed_download_count += 1
        elif line.startswith("AFFILIATION SECTION:"):
            affiliation_started = True
        elif line.startswith("-" * 10):
            continue
        elif current_paper and affiliation_started:
            stripped = line.strip()
            if stripped:
                paper_sections[current_paper].append(line.rstrip('\n'))

print(f"Successfully parsed {len(paper_sections)} papers.")
print(f"Total number of papers with download errors: {failed_download_count}")

# Save errors to separate file
with open('latex_error_papers.txt', 'w') as f:
    for paper in papers_with_errors:
        f.write(paper + "\n")
print("Saved titles with LaTeX errors to 'latex_error_papers.txt'")

# 3. Load the list of countries
print("Loading country list...")
list_countries = pd.read_csv('world_coords.csv')['country'].tolist()
list_countries.extend(['United States', 'United Kingdom', 'Canada', 'Australia', 'New Zealand', 'USA', 'UK', 'The Netherlands', 'China', 'South Korea',
                       'UAE','The United Arab Emirates', 'The United States', 'The United Kingdom', 'The United States of America', 'Italy', 'France', 'Germany', 'Spain', 'Japan'])
# Remove duplicates
list_countries = list(set(list_countries))
print(f"Loaded {len(list_countries)} countries.")

# 4. Search and extract lines
print("Matching and filtering papers...")
results = []
no_match_papers = []
error_titles_norm = set(t.strip().lower() for t in papers_with_errors)
normalized_sections = {k.strip().lower(): k for k in paper_sections.keys()}

for title in data['title']:
    matching_lines = []
    title_norm = str(title).strip().lower()
    
    is_error = title_norm in error_titles_norm
    found_in_latex = title_norm in normalized_sections
    
    if found_in_latex:
        original_key = normalized_sections[title_norm]
        section_lines = paper_sections[original_key]
        
        for l in section_lines:
            for country in list_countries:
                if country in l:
                    matching_lines.append(l.strip())
    
    # Track papers with no matches that aren't download errors
    if not is_error and len(matching_lines) == 0:
        no_match_papers.append(title)
        
    results.append({
        'title': title,
        'lines': matching_lines
    })

# 5. Save results
output_df = pd.DataFrame(results)
output_df.to_csv('extracted_affiliations.csv', index=False)
print(f"\nTask complete! Saved to 'extracted_affiliations.csv'.")
print(f"Found matches for {len(output_df[output_df['lines'].map(len) > 0])} papers.")

# Report on papers in 'data' that failed to download
data_error_count = sum(1 for t in data['title'] if str(t).strip().lower() in error_titles_norm)
print(f"Of the {len(data)} papers in your request, {data_error_count} had LaTeX download errors recorded.")

# Report on papers with no matches
print(f"Number of papers with NO country matches (excluding errors): {len(no_match_papers)}")
if no_match_papers:
    print("\nPapers with no country matches:")
    for p in no_match_papers[:10]: # Show first 10
        print(f"- {p}")
    if len(no_match_papers) > 10:
        print(f"... and {len(no_match_papers)-10} more.")

output_df.head()


Loading titles...
Loaded 3336 titles.
Parsing latex_affiliations_output.txt... this may take a moment.
Successfully parsed 10685 papers.
Total number of papers with download errors: 12917
Saved titles with LaTeX errors to 'latex_error_papers.txt'
Loading country list...
Loaded 253 countries.
Matching and filtering papers...

Task complete! Saved to 'extracted_affiliations.csv'.
Found matches for 1041 papers.
Of the 3336 papers in your request, 2152 had LaTeX download errors recorded.
Number of papers with NO country matches (excluding errors): 149

Papers with no country matches:
- Probabilistic mapping between multiparticle production variables and the depth of maximum in proton-induced extensive air showers
- The Leinster-Cobbold diversity index as a criterion for sub-clustering
- The Youngest Star Clusters in the Large Magellanic Cloud
- On the presence of a fifth force at the Galactic Center
- Heating, Excitation, Dissociation, and Ionization of Molecules by High-Energy Photons in 

,title,lines
0,Can a multi-tracer approach improve the constr...,[]
1,First large scale spatial and velocity pattern...,[]
2,A Stellar Magnesium to Silicon ratio in the at...,[]
3,Photometric and spectroscopic variability of b...,[]
4,Homogeneous measurements of proximity zone siz...,"[$^{1}$Leiden Observatory, Leiden University, ..."


In [67]:
import pandas as pd
import os

# 1. Load the titles to search for
print("Loading titles...")
with open('papers_with_missing_affiliations.txt', 'r') as f:
    titles = [line.strip() for line in f if line.strip()]
data = pd.DataFrame(titles, columns=['title'])
print(f"Loaded {len(data)} titles.")

# 2. Parse the latex affiliations file
print("Parsing latex affiliation sections... this may take a moment.")
paper_sections = {}
current_paper = None
affiliation_started = False
failed_download_count = 0
papers_with_errors = []

with open("latex_affiliations_output_2.txt", "r") as f:
    for line in f:
        if line.startswith("PAPER: "):
            current_paper = line.replace("PAPER: ", "").strip()
            paper_sections[current_paper] = []
            affiliation_started = False
        elif line.startswith("ERROR: Failed to download LaTeX sources"):
            if current_paper:
                papers_with_errors.append(current_paper)
                failed_download_count += 1
        elif line.startswith("AFFILIATION SECTION:"):
            affiliation_started = True
        elif line.startswith("-" * 10):
            continue
        elif current_paper and affiliation_started:
            stripped = line.strip()
            if stripped:
                paper_sections[current_paper].append(line.rstrip('\n'))

print(f"Successfully parsed {len(paper_sections)} papers.")
print(f"Total number of papers in output file with download errors: {failed_download_count}")

# 3. Load the list of countries
print("Loading country list...")
list_countries = pd.read_csv('world_coords.csv')['country'].tolist()
list_countries.extend(['United States', 'United Kingdom', 'Canada', 'Australia', 'New Zealand', 'USA', 'UK', 'The Netherlands', 'China', 'South Korea',
                       'UAE','The United Arab Emirates', 'The United States', 'The United Kingdom', 'The United States of America', 'Italy', 'France', 'Germany', 'Spain', 'Japan'])
# Remove duplicates
list_countries = list(set(list_countries))
print(f"Loaded {len(list_countries)} countries.")

# 4. Search and extract lines
print("Matching and filtering papers...")
results = []
no_match_papers = []
error_titles_norm = set(t.strip().lower() for t in papers_with_errors)
normalized_sections = {k.strip().lower(): k for k in paper_sections.keys()}

for title in data['title']:
    matching_lines = []
    title_norm = str(title).strip().lower()
    
    is_error = title_norm in error_titles_norm
    found_in_latex = title_norm in normalized_sections
    
    if found_in_latex:
        original_key = normalized_sections[title_norm]
        section_lines = paper_sections[original_key]
        
        for l in section_lines:
            for country in list_countries:
                if country in l:
                    matching_lines.append(l.strip())
    
    # Track papers with no matches that aren't download errors
    if not is_error and len(matching_lines) == 0:
        no_match_papers.append(title)
        
    results.append({
        'title': title,
        'lines': matching_lines
    })

# 5. Save results
output_df = pd.DataFrame(results)
output_df.to_csv('extracted_affiliations_2.csv', index=False)
print(f"\nTask complete! Saved to 'extracted_affiliations.csv'.")
print(f"Found matches for {len(output_df[output_df['lines'].map(len) > 0])} papers.")

# Report on papers in 'data' that failed to download
data_error_count = sum(1 for t in data['title'] if str(t).strip().lower() in error_titles_norm)
print(f"Of the {len(data)} papers in your request, {data_error_count} had LaTeX download errors recorded.")

# Report on papers with no matches
print(f"Number of papers with NO country matches (excluding errors): {len(no_match_papers)}")
if no_match_papers:
    print("\nPapers with no country matches:")
    for p in no_match_papers[:20]: # Show first 20
        print(f"- {p}")
    if len(no_match_papers) > 20:
        print(f"... and {len(no_match_papers)-20} more.")

output_df.head()


Loading titles...
Loaded 3336 titles.
Parsing latex affiliation sections... this may take a moment.
Successfully parsed 5974 papers.
Total number of papers in output file with download errors: 212
Loading country list...
Loaded 253 countries.
Matching and filtering papers...

Task complete! Saved to 'extracted_affiliations.csv'.
Found matches for 1519 papers.
Of the 3336 papers in your request, 126 had LaTeX download errors recorded.
Number of papers with NO country matches (excluding errors): 1691

Papers with no country matches:
- Can a multi-tracer approach improve the constraints on the turnover scale?
- Homogeneous measurements of proximity zone sizes for 59 quasars in the Epoch of Reionization
- The COSMOS-Web Lens Survey (COWLS) III: forecasts versus data
- JWST observations of segregated $^{12}$CO$_2$ and $^{13}$CO$_2$ ices in protostellar envelopes
- A direct black hole mass measurement in a Little Red Dot at the Epoch of Reionization
- Why the northern hemisphere needs a 30-4

,title,lines
0,Can a multi-tracer approach improve the constr...,[]
1,First large scale spatial and velocity pattern...,"[Universit\'e C\^ote d'Azur, Observatoire de l..."
2,A Stellar Magnesium to Silicon ratio in the at...,[\affil[1]{School of Earth and Space Explorati...
3,Photometric and spectroscopic variability of b...,"[\institute{Tartu Observatory, University of T..."
4,Homogeneous measurements of proximity zone siz...,[]


In [68]:
affils_1 = pd.read_csv('extracted_affiliations.csv')
affils_2 = pd.read_csv('extracted_affiliations_2.csv')

In [30]:
affils_1['lines'][0]

'[]'

In [69]:
counter_1 = 0
for i in range(len(affils_1)):
    line = affils_1['lines'][i]
    if len(line) > 0 and line != '[]':
        counter_1 += 1
print(counter_1)


1041


In [70]:
counter_2 = 0
for i in range(len(affils_2)):
    line = affils_2['lines'][i]
    if len(line) > 0 and line != '[]':
        counter_2 += 1
print(counter_2)


1519


In [71]:
total_counts = counter_1 + counter_2
total_papers = 3336
diff = total_papers - total_counts
print("Promlematic papers: ", diff)

Promlematic papers:  776


In [ ]:
# Number of Affiliations Found
affil_counter_1 = 0
for i in range(len(affils_1)):
    if 


0


In [74]:
index = 4
print(affils_1['lines'][index])
print(affils_1['title'][index])


['$^{1}$Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands\\\\', '$^{1}$Leiden Observatory, Leiden University, P.O. Box 9513, 2300 RA Leiden, The Netherlands\\\\', '$^{2}$Department of Physics, Broida Hall, University of California, Santa Barbara, Santa Barbara, CA 93106-9530, USA\\\\', '$^{3}$MIT Kavli Institute for Astrophysics and Space Research, Massachusetts Institute of Technology, Cambridge, MA 02139, USA']
Homogeneous measurements of proximity zone sizes for 59 quasars in the Epoch of Reionization


## Different Test

In [65]:
import re

# Method 2: Robust Regex-based author and affiliation extraction
print("Starting Method 2: Improved Regex-based extraction (Restored 'Final Fix')...")

# Regex to match \author, \affiliation, \alsoaffiliation, and \institute
# Pattern: \(author|affiliation|alsoaffiliation|institute)(?:\[[^\]]*\])?\s*\{((?:[^{}]|\{[^{}]*\})+)\}
combined_pattern = re.compile(r'\\(author|affiliation|alsoaffiliation|institute)(?:\[[^\]]*\])?\s*\{((?:[^{}]|\{[^{}]*\})+)\}')

results_method2 = []

for title in data['title']:
    author_list = []
    affil_list_of_lists = [] # Each element is a list of affiliations for the corresponding author
    
    title_norm = str(title).strip().lower()
    
    if title_norm in normalized_sections:
        original_key = normalized_sections[title_norm]
        section_lines = paper_sections[original_key]
        section_text = " ".join(section_lines)
        
        matches = combined_pattern.finditer(section_text)
        
        current_author_index = -1
        
        for match in matches:
            cmd_type = match.group(1)
            content = match.group(2).strip()
            
            # Clean up content (remove multiple spaces, newlines)
            content = re.sub(r'\s+', ' ', content)
            
            if cmd_type == 'author':
                author_list.append(content)
                affil_list_of_lists.append([])
                current_author_index += 1
            elif cmd_type in ('affiliation', 'institute', 'alsoaffiliation'):
                if current_author_index >= 0:
                    affil_list_of_lists[current_author_index].append(content)
        
    results_method2.append({
        'title': title,
        'authors': author_list,
        'affiliations': affil_list_of_lists
    })

# Create DataFrame and Save
output_df_method2 = pd.DataFrame(results_method2)
output_df_method2.to_csv('author_affiliations_method2.csv', index=False)

print(f"Extraction complete! Saved to 'author_affiliations_method2.csv'.")
print(f"Stats: Processed {len(output_df_method2)} papers.")
print(f"Papers with at least one author matched: {len(output_df_method2[output_df_method2['authors'].map(len) > 0])}")

output_df_method2.head()


Starting Method 2: Improved Regex-based extraction (Restored 'Final Fix')...
Extraction complete! Saved to 'author_affiliations_method2.csv'.
Stats: Processed 3336 papers.
Papers with at least one author matched: 823


,title,authors,affiliations
0,Can a multi-tracer approach improve the constr...,[],[]
1,First large scale spatial and velocity pattern...,[],[]
2,A Stellar Magnesium to Silicon ratio in the at...,[],[]
3,Photometric and spectroscopic variability of b...,[],[]
4,Homogeneous measurements of proximity zone siz...,[Silvia Onorato$^{1}$\thanks{E-mail: onorato@s...,[[]]


In [54]:
output_df_method2

,title,authors,affiliations
0,Can a multi-tracer approach improve the constr...,[],[]
1,First large scale spatial and velocity pattern...,[],[]
2,A Stellar Magnesium to Silicon ratio in the at...,[],[]
3,Photometric and spectroscopic variability of b...,[],[]
4,Homogeneous measurements of proximity zone siz...,[],[]
...,...,...,...
3331,CubeSat Orbit Insertion Maneuvering Using J2 P...,[],[]
3332,New Bulgarian-Austrian project 'Joint observat...,[],[]
3333,Year six photometric measurements of known Tra...,[],[]
3334,"Alfvén wave propagation, reflection and trappi...",[],[]
